In [ ]:
# Use historical offensive MLB data by player to predict individual player OPS for 2021.
# Using OPS to Salary to measure value of a player, make a team roster of the best value players

In [ ]:
## Data preparation
# 1. Rolling annual window
# 80% training data 30% testing data
# 3. Use MinMaxScaler to X and y values

In [3]:
# Imports

import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import glob

In [4]:
# Load player stats

path = '../Resources/batting_stats_*.csv'
all_files = glob.glob(path)
all_files

['../Resources\\batting_stats_2010.csv',
 '../Resources\\batting_stats_2011.csv',
 '../Resources\\batting_stats_2012.csv',
 '../Resources\\batting_stats_2013.csv',
 '../Resources\\batting_stats_2014.csv',
 '../Resources\\batting_stats_2015.csv',
 '../Resources\\batting_stats_2016.csv',
 '../Resources\\batting_stats_2017.csv',
 '../Resources\\batting_stats_2018.csv',
 '../Resources\\batting_stats_2019.csv',
 '../Resources\\batting_stats_2020.csv',
 '../Resources\\batting_stats_2021.csv']

In [6]:
# Turn into dataframe
data = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)    
    year = file[-8:-4]
    df['Year'] = year
    data.append(df)

In [7]:
# Concatenate dataframes

stats_df = pd.concat(data, axis=0, ignore_index=True)

In [8]:
stats_df

,Player,Team,Pos,Age,G,AB,R,H,2B,3B,...,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS,Year
0,Ichiro Suzuki,SEA,OF,47,162,680,74,214,30,3,...,45,86,3,1,3,0.315,0.359,0.394,0.753,2010
1,Derek Jeter,NYY,SS,47,157,663,111,179,30,3,...,63,106,1,3,9,0.270,0.340,0.370,0.710,2010
2,Michael Young,TEX,3B,44,157,656,99,186,36,3,...,50,115,0,11,1,0.284,0.330,0.444,0.774,2010
3,Juan Pierre,CWS,OF,43,160,651,96,179,18,3,...,45,47,15,2,21,0.275,0.341,0.316,0.657,2010
4,Rickie Weeks,MIL,DH,38,160,651,112,175,32,4,...,76,184,0,2,25,0.269,0.366,0.464,0.830,2010
5,Marco Scutaro,BOS,2B,45,150,632,92,174,38,0,...,53,71,4,3,3,0.275,0.333,0.388,0.721,2010
6,Nick Markakis,BAL,OF,37,160,629,79,187,45,3,...,73,93,0,5,2,0.297,0.370,0.436,0.806,2010
7,Denard Span,MIN,OF,37,153,629,85,166,24,10,...,60,74,10,2,4,0.264,0.331,0.348,0.679,2010
8,Brandon Phillips,CIN,2B,40,155,626,100,172,33,5,...,46,83,6,1,8,0.275,0.332,0.430,0.762,2010
9,Robinson Cano,NYY,2B,38,160,626,103,200,41,3,...,57,77,0,5,8,0.319,0.381,0.534,0.915,2010


In [9]:
# Show columns

stats_df.columns

Index(['Player', 'Team', 'Pos', 'Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'SH', 'SF', 'HBP', 'AVG', 'OBP', 'SLG',
       'OPS', 'Year'],
      dtype='object')

In [11]:
# Select relevant columns

stats_df = stats_df[['Year', 'Player', 'Team', 'Pos', 'Age', 'G', 'AB', 'AVG', 'OBP', 'SLG','OPS']]
stats_df.head()

,Year,Player,Team,Pos,Age,G,AB,AVG,OBP,SLG,OPS
0,2010,Ichiro Suzuki,SEA,OF,47,162,680,0.315,0.359,0.394,0.753
1,2010,Derek Jeter,NYY,SS,47,157,663,0.270,0.340,0.370,0.710
2,2010,Michael Young,TEX,3B,44,157,656,0.284,0.330,0.444,0.774
3,2010,Juan Pierre,CWS,OF,43,160,651,0.275,0.341,0.316,0.657
4,2010,Rickie Weeks,MIL,DH,38,160,651,0.269,0.366,0.464,0.830


In [ ]:
# Roster positions - we need to find the best OPS for salary value per player

# 1 (pitcher) = P
# 2 (catcher) = C
# 3 (first baseman) = 1B
# 4 (second baseman) = 2B
# 5 (third baseman) = 3B
# 6 (shortstop) = SS
# 7 (left fielder) = LF
# 8 (center fielder) = CF
# 9 (right fielder) = RF

In [12]:
# Below function takes column number for the features (X) and target (y)
# It accumulates the data with a rolling window of X(t-n) to predict Xt
# Finally, it returns a numpy array of X and y

def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    print(len(df) - window - 1)
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [ ]:
# Predict Closing Prices using a 10 day window of previous closing prices

window_size = 1

# Column index 0 is the 'index' column
# Column index 1 is the 'stock' column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)